In [5]:
import os
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI
import dotenv
import numpy as np

dotenv.load_dotenv()



client = OpenAI(api_key=os.getenv("OPENAI_API_KEY_PERSONAL"))

df_products = pd.read_json('../data/sample_data_with_embeddings.jsonl', orient='records', lines=True)    

In [6]:
class QueryList(BaseModel):
    queries: list[str]

def expand_query(query):
        response = client.beta.chat.completions.parse(
            model=os.getenv("LLM_MODEL"),
            messages=[
                {
                    "role": "system",
                    "content": """
                                You are a helpful assistant that expands queries. 
                                Based on the user query, you will expand the query to include more relevant products. 
                                Your output will be used to do a similarity search on a product database.
                                Return a list of expanded queries with only three queries.
                                """,
                },
                {"role": "user", "content": query},
            ],
            response_format=QueryList,
        )
        return response.choices[0].message.parsed

In [37]:
original_query = "I am looking for an outfit for a wedding for me. I am a groomsman. Adult size."

queries = expand_query(original_query).queries

In [38]:
def get_embeddings(input):
    print(input)
    response = client.embeddings.create(input=input, model=os.getenv("EMBEDDING_MODEL"))
    return [data.embedding for data in response.data]

embeddings = get_embeddings(queries)

['groomsman suit wedding outfit adult size', 'groomsman tuxedo formal wear', 'matching accessories for groomsman outfit']


In [39]:
def find_similar_products(
    query_embeddings: list[list[float]], top_k: int = 5
):
    product_embeddings = np.array(list(df_products["embedding"].apply(lambda x: list(x))))
    # Calculate cosine similarity between query embeddings and product embeddings
    similarities = np.dot(query_embeddings, product_embeddings.T)

    # Get the top 5 most similar products for each query
    top_indices = np.argsort(similarities, axis=1)[:, -1 * top_k :]

    top_indices = top_indices.ravel()

    # Get the products for the top indices
    products = df_products.iloc[top_indices]

    return products

In [40]:
product_results = find_similar_products(embeddings)

In [41]:
# Filter out products with average rating less than 3
def filter_products(products):
  return products[products["average_rating"] >= 3]

product_results = filter_products(product_results)

In [42]:
product_results

,title,average_rating,rating_number,features,price,store,thumbnail,embedding
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."
3537,"Black Bow Tie, Cummerbund and Cufflinks & Stud...",4.5,5,"[Satin, Drawstring closure, HIGH QUALITY SOFT ...",33.99,S.H. Churchill & Co.,https://m.media-amazon.com/images/I/318KU0C+oY...,"[0.0157968514, 0.0562614575, -0.0178959817, 0...."
3437,Little Gents Boys Suits Dark Grey - Kids Suit ...,4.0,7,"[Cotton Blend, Button closure, YOUR SON WILL L...",46.99,Little Gents,https://m.media-amazon.com/images/I/3193Wv4k57...,"[-0.0163080767, 0.0172005594, -0.0390055142, -..."
620,Tailorsun Tweed Vest Vintage Rustic Wedding Ve...,4.2,25,"[Imported, Button closure, Sizes are now corre...",29.00,Tailorsun,https://m.media-amazon.com/images/I/51L65WniXC...,"[-0.012405319100000001, 0.0764714554, -0.04585..."
620,Tailorsun Tweed Vest Vintage Rustic Wedding Ve...,4.2,25,"[Imported, Button closure, Sizes are now corre...",29.00,Tailorsun,https://m.media-amazon.com/images/I/51L65WniXC...,"[-0.012405319100000001, 0.0764714554, -0.04585..."
717,Epoint Men's Fashion Paisley Microfiber Dress ...,4.4,156,"[100% Woven Microfiber, Drawstring closure, Th...",22.93,Epoint,https://m.media-amazon.com/images/I/41pMZ8C9EJ...,"[0.009126815, -0.0051772515000000005, -0.05662..."
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."
3479,Sportoli Men's Formal Satin Adjustable Solid P...,3.9,12,[Satin],6.79,Sportoli,https://m.media-amazon.com/images/I/31fGBJrN3e...,"[-0.0165207591, 0.0515770055, -0.0199793726, 0..."
1037,"Tuxedo Shirt, Cummerbund, Bow Tie, Cufflink & ...",3.6,25,[Drawstring closure],26.00,Neil Allyn,https://m.media-amazon.com/images/I/41IUCjYr2y...,"[0.0013640962, 0.0185012035, -0.0066230986, 0...."
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."


In [43]:
import base64
import requests

def get_and_encode_image(image_url):
    response = requests.get(image_url)
    return base64.b64encode(response.content).decode('utf-8')


In [48]:
from pydantic import BaseModel

class ProductValidationResponse(BaseModel):
    answer: bool
    reason: str


def validate_product_with_query(query, product_title, product_image_base64):
    response = client.beta.chat.completions.parse(
        model=os.getenv("LLM_MODEL"),
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": f""" You are a fashion expert analyzing a clothing item.
                            You will be shown an image of a clothing item and given a text query.
                            Evaluate if this item matches the description in the query.
                            If the user was specific about what they were looking for, make sure the item matches that description.
                            Example: If the user is looking for a "mens wedding outfit", make sure the item is for an adult male and not a child.
                            
                            Provide your analysis in JSON format with two fields:
                            - "answer": Must be "True" or "False" indicating if the item matches the query
                            - "reason": A concise explanation of your decision if the item matches the query. Leave blank if the item does not match the query.
                            
                            Do not describe the item itself. Focus only on its relevance to the query.
                            
                            <query>
                            {query}
                            </query>

                            <product_title>
                            {product_title}
                            </product_title>
                            """,
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{product_image_base64}",
                },
                },
            ],
            }
        ],
        response_format=ProductValidationResponse,
    )

    return response.choices[0].message.parsed



In [49]:

product = product_results.iloc[0]

product_image_base64 = get_and_encode_image(product["thumbnail"])

validate_product_with_query(original_query, product["title"], product_image_base64)

ProductValidationResponse(answer=True, reason="The item is an adult men's tuxedo vest and necktie set, suitable for a groomsman at a wedding.")

In [ ]:
def validate_single_product(product_row):
    """
    Validates a single product against the original query using its thumbnail image.
    Returns the validation response.
    """
    product_image_base64 = get_and_encode_image(product_row["thumbnail"])
    return validate_product_with_query(original_query, product_row["title"], product_image_base64)

# Filter products to only those that match the query according to the vision model
validated_products = product_results[
    product_results.apply(validate_single_product, axis=1).map(lambda x: x.answer)
]

validated_products

,title,average_rating,rating_number,features,price,store,thumbnail,embedding
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."
620,Tailorsun Tweed Vest Vintage Rustic Wedding Ve...,4.2,25,"[Imported, Button closure, Sizes are now corre...",29.00,Tailorsun,https://m.media-amazon.com/images/I/51L65WniXC...,"[-0.012405319100000001, 0.0764714554, -0.04585..."
620,Tailorsun Tweed Vest Vintage Rustic Wedding Ve...,4.2,25,"[Imported, Button closure, Sizes are now corre...",29.00,Tailorsun,https://m.media-amazon.com/images/I/51L65WniXC...,"[-0.012405319100000001, 0.0764714554, -0.04585..."
717,Epoint Men's Fashion Paisley Microfiber Dress ...,4.4,156,"[100% Woven Microfiber, Drawstring closure, Th...",22.93,Epoint,https://m.media-amazon.com/images/I/41pMZ8C9EJ...,"[0.009126815, -0.0051772515000000005, -0.05662..."
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."
1037,"Tuxedo Shirt, Cummerbund, Bow Tie, Cufflink & ...",3.6,25,[Drawstring closure],26.00,Neil Allyn,https://m.media-amazon.com/images/I/41IUCjYr2y...,"[0.0013640962, 0.0185012035, -0.0066230986, 0...."
799,Epoint Men's Fashion Marriage Paisley Microfib...,4.1,37,"[100% Woven Microfiber, Drawstring closure, Th...",24.99,Epoint,https://m.media-amazon.com/images/I/41cFdjEAtm...,"[0.00313825, 0.0041489243000000006, -0.0520087..."


In [ ]:
def generate_natural_language_response(validated_products, original_query):
    """
    Generates a natural language response summarizing the validated product results.
    
    Args:
        validated_products (pd.DataFrame): DataFrame containing the validated products
        original_query (str): The original search query
        
    Returns:
        str: A natural language response describing the search results
    """
    messages = [
        {
            "role": "system",
            "content": """You are a helpful shopping assistant. Summarize the search results in a natural, 
            conversational way. Include key details like number of results, price ranges if available, 
            and notable brands or features. Avoid any negative language when describing the products.

            Do not list the number of products that you are showing. Mention that you have found a few options that you think will work.
            encourage the user to look at the products and see if they like any of them and say there are more options if they want to see more.
            """
        },
        {
            "role": "user", 
            "content": f"""Here is a search query: {original_query}
            
            Here are the matching products:
            {validated_products.to_string()}
            
            Please provide a natural language summary of these results."""
        }
    ]
    
    response = client.chat.completions.create(
        model=os.getenv("LLM_MODEL"),
        messages=messages,
        temperature=0.5,
    )
    
    return response.choices[0].message.content

def convert_results_to_json(validated_products):
    """
    Converts the validated products DataFrame to a JSON format suitable for API responses.
    
    Args:
        validated_products (pd.DataFrame): DataFrame containing the validated products
        
    Returns:
        dict: JSON-compatible dictionary containing the product results
    """
    products_list = []
    
    for _, row in validated_products.iterrows():
        product_dict = {
            "title": row["title"],
            "brand": row["brand"],
            "thumbnail": row["thumbnail"],
            "average_rating": float(row["average_rating"]),
            "embedding": row["embedding"].tolist() if isinstance(row["embedding"], (list, np.ndarray)) else None
        }
        products_list.append(product_dict)
    
    return {
        "total_results": len(products_list),
        "products": products_list
    }

# Example usage:
natural_response = generate_natural_language_response(validated_products, original_query)
# json_results = convert_results_to_json(validated_products)

print("Natural Language Response:")
print(natural_response)
# print("\nJSON Results:")
# print(json_results)


Natural Language Response:
I found some great options for your groomsman outfit for the wedding! There are a total of **five products** that might suit your needs, with prices ranging from **$22.93 to $29.00**.

1. **Epoint Men's Fashion Marriage Paisley Microfiber Tuxedo Vest Necktie Set** - This stylish set is priced at **$24.99** and has an average rating of **4.1** from 37 reviews. It features a 100% woven microfiber material and comes with a necktie, making it a versatile choice for various occasions including weddings.

   ![Epoint Tuxedo Vest](https://m.media-amazon.com/images/I/41cFdjEAtmL._AC_SR38,50_.jpg)

2. **Tailorsun Tweed Vest Vintage Rustic Wedding Vest** - Priced at **$29.00**, this vest has a **4.2** rating from 25 reviews. It's designed for a rustic wedding vibe and features three pockets and fabric buttons. It's important to check the size chart as it's a slim fit.

   ![Tailorsun Tweed Vest](https://m.media-amazon.com/images/I/51L65WniXCL._AC_SR38,50_.jpg)

3. **Ep